# CelebA + Adversarial Robustness Toolbox Demo

This notebook:
1. Installs necessary packages.
2. Loads the CelebA dataset from Hugging Face.
3. Trains a simple CNN.
4. Demonstrates a FGSM attack.
5. Shows an example of face obfuscation with OpenCV.

Please ensure you have **accepted the license** for `celeba` on Hugging Face and that you're **logged in** with a valid Hugging Face token.


In [ ]:
%%capture
# Install packages
!pip install --upgrade pip
!pip install --upgrade datasets huggingface_hub
!pip install adversarial-robustness-toolbox opencv-python

## (Optional) Hugging Face Authentication
If CelebA is public and you have accepted the terms, you *might* be able to skip. If you see `DatasetNotFoundError`, come back here.


In [ ]:
# (Optional) authenticate with Hugging Face
from huggingface_hub import notebook_login
# notebook_login()  # Uncomment if needed; paste your token

## Imports

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import cv2
import random

from art.estimators.classification import TensorFlowV2Classifier
from art.attacks.evasion import FastGradientMethod
from art.utils import load_dataset

from datasets import load_dataset
from PIL import Image

matplotlib.rcParams.update({"font.size": 14})

## Load the CelebA dataset
We use `.with_format('image')` so that each row's `"image"` key is a PIL image (instead of a string or array).

In [ ]:
celeba_all = load_dataset("celeba", "original")  # Requires having accepted the license
celeba_all = celeba_all.with_format("image")

print(celeba_all)
print("\nSample record:", celeba_all["train"][0])

### Create a small subset for demonstration
We’ll take 2k images for training, 500 images for test to keep it short. Adjust as needed.


In [ ]:
train_data = celeba_all["train"].select(range(2000))
test_data = celeba_all["test"].select(range(500))

print(train_data[0].keys())  # should have 'image', 'attributes', etc.


## Preprocess data (resize to 28×28, grayscale, pick a 'Male' label)

In [ ]:
def transform_celebA_entry(entry, image_size=(28, 28)):
    """
    - entry["image"]: a PIL image.
    - entry["attributes"]: dict with many attributes, including 'Male'.
    """
    # Convert to grayscale, resize
    img = entry["image"].convert("L").resize(image_size)
    arr = np.array(img, dtype=np.float32) / 255.0
    # 'Male' is True/False in attributes
    is_male = bool(entry["attributes"]["Male"])
    one_hot = np.zeros(2, dtype=np.float32)
    if is_male:
        one_hot[1] = 1.0  # male
    else:
        one_hot[0] = 1.0  # female
    return arr, one_hot

train_images = []
train_labels = []
for i in range(len(train_data)):
    entry = train_data[i]
    arr, label = transform_celebA_entry(entry)
    train_images.append(arr)
    train_labels.append(label)

test_images = []
test_labels = []
for i in range(len(test_data)):
    entry = test_data[i]
    arr, label = transform_celebA_entry(entry)
    test_images.append(arr)
    test_labels.append(label)

train_images = np.array(train_images).reshape((-1, 28, 28, 1))
train_labels = np.array(train_labels)
test_images = np.array(test_images).reshape((-1, 28, 28, 1))
test_labels = np.array(test_labels)

print("Train images:", train_images.shape)
print("Train labels:", train_labels.shape)
print("Test images:\t", test_images.shape)
print("Test labels:\t", test_labels.shape)

## Train a simple CNN

In [ ]:
def create_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
        tf.keras.layers.MaxPool2D(2,2),
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPool2D(2,2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(2, activation='softmax')
    ])
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

model = create_model()
model.fit(
    train_images, train_labels,
    epochs=3,
    batch_size=64
)

## FGSM Attack with ART

In [ ]:
loss_object = tf.keras.losses.CategoricalCrossentropy()

classifier = TensorFlowV2Classifier(
    clip_values=(0, 1),
    model=model,
    nb_classes=2,
    input_shape=(28,28,1),
    loss_object=loss_object,
)

fgsm = FastGradientMethod(estimator=classifier, eps=0.3)
test_images_adv = fgsm.generate(x=test_images)

score_clean = model.evaluate(test_images, test_labels, verbose=0)
score_adv   = model.evaluate(test_images_adv, test_labels, verbose=0)

print("Clean test accuracy:      {:.2f}%".format(score_clean[1]*100))
print("Adversarial test accuracy: {:.2f}%".format(score_adv[1]*100))

## Face Obfuscation Demo
We can optionally do face obfuscation with Haar Cascade. We'll pick random face patches from the CelebA train subset. Just a minimal example!

In [ ]:
# Download Haar Cascade for frontal faces
!wget -q https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def random_face_from_celeba():
    random_idx = random.randint(0, len(train_data)-1)
    # This is a PIL image in RGB
    return train_data[random_idx]['image'].convert('RGB')

def obfuscate_face(image_path):
    """Detect faces, replace with random CelebA face patches."""
    img_bgr = cv2.imread(image_path)
    if img_bgr is None:
        print("Could not read image.")
        return None

    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        # get random celeb face
        celeb_pil = random_face_from_celeba()
        # resize to face bounding box
        celeb_pil = celeb_pil.resize((w,h))
        celeb_np = np.array(celeb_pil)  # shape (h, w, 3)
        # Overwrite region in original image
        # Note: celeb_np is RGB, but img_bgr is BGR
        img_bgr[y:y+h, x:x+w] = celeb_np[:, :, ::-1]

    return img_bgr

print("Face obfuscation ready.")

### Upload an image (in Colab) and show obfuscation result

In [ ]:
from google.colab import files

uploaded = files.upload()  # pick an image with a face
if uploaded:
    up_name = list(uploaded.keys())[0]
    result_bgr = obfuscate_face(up_name)
    if result_bgr is not None:
        result_rgb = cv2.cvtColor(result_bgr, cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(6,6))
        plt.imshow(result_rgb)
        plt.title("Obfuscated face w/ random CelebA patch")
        plt.axis('off')
        plt.show()